### Importing the libraries

In [1]:
import pydicom
import numpy as np
import pandas as pd
from PIL import Image

In [2]:
dicom_img = pydicom.dcmread("data/sample.dcm")

In [3]:
dicom_img 

Dataset.file_meta -------------------------------
(0002,0000) File Meta Information Group Length  UL: 206
(0002,0001) File Meta Information Version       OB: b'\x00\x01'
(0002,0002) Media Storage SOP Class UID         UI: CT Image Storage
(0002,0003) Media Storage SOP Instance UID      UI: 1.3.6.1.4.1.14519.5.2.1.6279.6001.146096930167080421473717909599
(0002,0010) Transfer Syntax UID                 UI: Explicit VR Little Endian
(0002,0012) Implementation Class UID            UI: 1.3.6.1.4.1.22213.1.143
(0002,0013) Implementation Version Name         SH: '0.5'
(0002,0016) Source Application Entity Title     AE: 'POSDA'
-------------------------------------------------
(0008,0005) Specific Character Set              CS: 'ISO_IR 100'
(0008,0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY', 'AXIAL']
(0008,0016) SOP Class UID                       UI: CT Image Storage
(0008,0018) SOP Instance UID                    UI: 1.3.6.1.4.1.14519.5.2.1.6279.6001.14609693016708042

In [4]:
#Reading any metadata header 
id = dicom_img.PatientID
name = dicom_img.PatientName
print("Name : ",name, "ID : ",id)

Name :   ID :  LIDC-IDRI-0012


In [5]:
dicom_array = dicom_img.pixel_array
print(dicom_array)

[[-1024 -1024 -1024 ... -1024 -1024 -1024]
 [-1024 -1024 -1024 ... -1024 -1024 -1024]
 [-1024 -1024 -1024 ... -1024 -1024 -1024]
 ...
 [-1024 -1024 -1024 ... -1024 -1024 -1024]
 [-1024 -1024 -1024 ... -1024 -1024 -1024]
 [-1024 -1024 -1024 ... -1024 -1024 -1024]]


In [6]:
dicom_array.shape

(512, 512)

## Show Image

In [7]:
type(dicom_array)
np.unique(dicom_array) 

array([-1024,     0,     1, ...,  2149,  2175,  2208], dtype=int16)

In [8]:
#convert the int 16  to a numpy array float numbers
float_dicom_array = dicom_array.astype(float)
#in the dicom array we see negative values in the array it removes the negative values and convert to postitive. maxumun function takes 2 arg is the the pixel array another is the value we want to set , for this array we have set the set the negative values to become 0
positive_dicom_array = np.maximum(float_dicom_array,0)

In [9]:
np.unique(positive_dicom_array)

array([0.000e+00, 1.000e+00, 2.000e+00, ..., 2.149e+03, 2.175e+03,
       2.208e+03])

In [10]:
#we are normalizing the value of positive_dicom_array by deviding the whole array with its maximum number so the values of every idex in the array is between 0-1 , hence normalized
normalized_dicom_array = positive_dicom_array/positive_dicom_array.max()
np.unique(normalized_dicom_array)

array([0.00000000e+00, 4.52898551e-04, 9.05797101e-04, ...,
       9.73278986e-01, 9.85054348e-01, 1.00000000e+00])

In [11]:
normalized_dicom_array = normalized_dicom_array * 255.0
np.unique(normalized_dicom_array)

array([0.00000000e+00, 1.15489130e-01, 2.30978261e-01, ...,
       2.48186141e+02, 2.51188859e+02, 2.55000000e+02])

In [12]:
uint8_image = np.uint8(normalized_dicom_array)
np.unique(uint8_image)

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [ ]:
pillow_image = Image.fromarray(uint8_image)
# pillow_image.show()

## Show Image 2 using hounsfiled unit

In [18]:
hounsfield_min = np.min(dicom_array)
hounsfield_max = np.max(dicom_array)
hounsfield_range = hounsfield_max - hounsfield_min
print("hounsfield min : ", hounsfield_min)
print("hounsfield max : ", hounsfield_max)
print("hounsfield range : ",hounsfield_range)

hounsfield min :  -1024
hounsfield max :  2208
hounsfield range :  3232


In [22]:
dicom_array[dicom_array<hounsfield_min] = hounsfield_min
dicom_array[dicom_array>hounsfield_max] = hounsfield_max
hounsfield_normalized_array = (dicom_array - hounsfield_min) / hounsfield_range
hounsfield_normalized_array = hounsfield_normalized_array * 255.0
hounsfield_uint8_image = np.uint8(hounsfield_normalized_array)

In [23]:
hounsfield_pillow_image = Image.fromarray(hounsfield_uint8_image)
hounsfield_pillow_image.show()